# Primeiros passos com PySpark

Neste Objeto de Aprendizagem daremos nossos primeiros passos com o PySpark e Spark Dataframes. O objetivo aqui é conhecer os principais objetos do PySpark e introduzir os métodos mais básicos para familiarizar com a tecnologia.

Começaremos pela importação do pacote do PySpark que engloba as operações com DataFrames e então criaremos um pequeno DataFrame que será utilizado nos exemplos.

### Bibliotecas necessárias

Por enquanto precisaremos somente do módulo `pyspark.sql`. O pacote PySpark possui diversos módulos, mas por enquanto precisaremos somente dos objetos que estão em `pyspark.sql`.

In [3]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [5]:
# Uso do Spark Dataframes no PySpark
from pyspark.sql import *

### Conectando com o Spark

O próximo passo é iniciar uma sessão do Spark (`SparkSession`), cujo papel é o de comunicação com o Cluster. No exemplo abaixo criaremos uma sessão local - ou seja, com um minicluster na sua própria máquina. Esta sessão local é definida por meio do método `master`. O método `master` indica qual o tipo de Cluster onde conectaremos e outros detalhes. 

No nosso caso indicamos que o tipo de Cluster é local e que este utilizaremos 2 processadores para execução das tarefas do Spark. Percebam que mesmo em modo local temos à nossa disposição a capacidade de processamento _multicore_.

**IMPORTANTE**: No ambiente da **Databricks** não precisamos criar uma Sessão pois o notebook será vinculado (_attached_) a um cluster no momento da execução. Logo, o bloco de código abaixo não será necessário **na Databricks**.

In [ ]:
# Vamos trabalhar com o Spark localmente, sem o uso de um cluster.
#spark = SparkSession \
#    .builder \
#    .master("local[2]") \
#    .appName("Primeiros passos") \
#    .getOrCreate()

## Criação de um Data Frame via código

Nesta sessão criaremos um DataFrame diretamente via código. Esta não é uma prática muito comum, visto que trabalhamos com grandes volumes de dados obtidos através de Sistemas de Armazenamento. 

Mas não pensem que a criação de um DataFrame via código só serve para exemplos! Ainda veremos casos onde esta prática ajudará na resolução de problemas!

### Exemplo Prático

Nosso exemplo prático utiliza a relação de disciplinas que compõe esta especialização e suas cargas horárias!

Os dados para este exemplo foram obtidos **manualmente** da página de Estrutura Curricular do curso, disponível [aqui](http://www.unisinos.br/especializacao/big-data-data-science-e-data-analytics/ead/sao-leopoldo/estrutura-curricular).

#### Definição da estrutura: Registros e Colunas

Para criar um DataFrame por código precisamos inicialmente definir sua estrutura. A linha de código abaixo define que nosso DataFrame será formado por disciplinas, onde cada registro (**Row**) será uma disciplina. Os atributos de uma disciplina disponíveis serão o _nome_ e a _carga horária_.

In [6]:
# Estrutura do nosso DataFrame
Disciplina = Row("nome", "carga_horaria")

#### Criação de instâncias (registros e atributos)

Nosso próximo passo é a criação de instâncias para popular o DataFrame. Usaremos a estrutura `Disciplina` recém criada para instanciar cada uma das disciplinas da especialização e sua carga horária. 

Neste exemplo foi criada uma referência (`d01` a `d14`) por disciplina para deixar o código mais claro. No passo seguinte criaremos uma lista que agrupará todas as disciplinas e servirá de fonte para envio dos dados ao Spark.

In [7]:
# Cada uma das disciplinas da especialização é criada como uma instância do registro Disciplina.

d01 = Disciplina("Introdução a BigData e Analytics", 36)
d02 = Disciplina("Estatística aplicada", 24)
d03 = Disciplina("Visualização de dados e informação", 24)
d04 = Disciplina("Compartilhamento e segurança de dados", 24)
d05 = Disciplina("Introdução a Python e linguagem R", 36)
d06 = Disciplina("Machine Learning", 24)
d07 = Disciplina("Processamento de Alto Desempenho e Aplicações", 24)
d08 = Disciplina("Lidando com BigData: Apache Spark, Hadoop, MapReduce, Hive", 24)
d09 = Disciplina("Gerenciamento e Processamento de grande volume de dados", 24)
d10 = Disciplina("Internet das Coisas e Aplicações Distribuídas", 24)
d11 = Disciplina("Deep Learning", 24)
d12 = Disciplina("Business Intelligence e BigData", 24)
d13 = Disciplina("Atividades Integradoras", 12)
d14 = Disciplina("Preparação para Projeto Aplicado", 36)

Por meio da função `display` temos uma prévia do que será nosso DataFrame! Atentem para o fato de que até aqui nossos dados estão no Python e não no Spark. Ainda não temos um DataFrame!

No ambiente **Databricks** o resultado da função `display` será apresentado de forma mais amigável pois o mecanismo de notebooks do ambiente está preparado para formatação dos objetos Row do Spark. Lá a visualização da lista `Row` e posteriormente do DataFrame serão muito parecidas!

In [8]:
especializacao_bigdata_datascience = [d01, d02, d03, d04, d05, d06, d07, d08, d09, d10, d11, d12, d13, d14]

display(especializacao_bigdata_datascience)

[Row(nome='Introdução a BigData e Analytics', carga_horaria=36),
 Row(nome='Estatística aplicada', carga_horaria=24),
 Row(nome='Visualização de dados e informação', carga_horaria=24),
 Row(nome='Compartilhamento e segurança de dados', carga_horaria=24),
 Row(nome='Introdução a Python e linguagem R', carga_horaria=36),
 Row(nome='Machine Learning', carga_horaria=24),
 Row(nome='Processamento de Alto Desempenho e Aplicações', carga_horaria=24),
 Row(nome='Lidando com BigData: Apache Spark, Hadoop, MapReduce, Hive', carga_horaria=24),
 Row(nome='Gerenciamento e Processamento de grande volume de dados', carga_horaria=24),
 Row(nome='Internet das Coisas e Aplicações Distribuídas', carga_horaria=24),
 Row(nome='Deep Learning', carga_horaria=24),
 Row(nome='Business Intelligence e BigData', carga_horaria=24),
 Row(nome='Atividades Integradoras', carga_horaria=12),
 Row(nome='Preparação para Projeto Aplicado', carga_horaria=36)]

#### Criação do DataFrame por meio da transferência dos dados da lista

Lembram que mais acima eu descrevi o `SparkSession` como o canal de comunicação com o Cluster? Pois bem, agora veremos na prática o que isso significa. Nossa sessão possibilita a criação de um DataFrame pelo método `createDataFrame`. Este método:
- envia a lista de objetos `Row` para o Cluster
- cria uma estrutra de DataFrame no Cluster
- popula o DataFrame com os objetos `Row` recebidos
- retorna a referência ao DataFrame para o Python

In [13]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()
df_especializacao = spark.createDataFrame(especializacao_bigdata_datascience)

Voltaremos a usar a função `display`, desta vez para inspecionar o conteúdo da referência ao DataFrame que o método `createDataFrame` retornou para nós. Aqui percebemos que se trata de um DataFrame, e que ele possui duas colunas:

- nome: string
- carga_horaria: bigint

**Importante**: Na **Databricks** a função `display` exibe o conteúdo do nosso DataFrame de forma bastante similar a quando usamos `display` para visualizar o conteúdo da lista de objetos `Row`.

In [14]:
display(df_especializacao)

DataFrame[nome: string, carga_horaria: bigint]

### Visualização de dados de um DataFrame

#### Método `show`

O método `show` exibe registros do DataFrame formatados em modo texto. Se a chamada ao método for sem nenhum parâmetro ele retornará uma tabela com os nomes das coluns em cabeçalho, registros até um máximo de 20 linhas e os valores das colunas de tipo String (texto) serão exibidos até um máximo de 20 caracteres.

A documentação do método `show` ([link](http://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.show)) detalha os seguintes parâmetros:

- **n** – Número de registros a exibir. Se quisermos uma quantidade diferente de 20 registros então devemos informar a quantidade neste parâmetro.
- **truncate** – Se 20 caracteres for pouco (e no nosso exemplo vimos que é pouco) então devemos informar quantos caracteres das colunas String devem ser mostrados. Se o DataFrame tiver muitas colunas do tipo String a visualização pode ficar difícil.
- **vertical** – Se for False (padrão), exibe em formato de tabela. Se for True, exibirá cada coluna em uma linha, em formato de lista de valores.

In [15]:
df_especializacao.show()

+--------------------+-------------+
|                nome|carga_horaria|
+--------------------+-------------+
|Introdução a BigD...|           36|
|Estatística aplicada|           24|
|Visualização de d...|           24|
|Compartilhamento ...|           24|
|Introdução a Pyth...|           36|
|    Machine Learning|           24|
|Processamento de ...|           24|
|Lidando com BigDa...|           24|
|Gerenciamento e P...|           24|
|Internet das Cois...|           24|
|       Deep Learning|           24|
|Business Intellig...|           24|
|Atividades Integr...|           12|
|Preparação para P...|           36|
+--------------------+-------------+



In [16]:
# Lista de registros, exibindo os primeiros 60 caracteres de cada nome.
df_especializacao.show(vertical=True, truncate=60)

-RECORD 0-------------------------------------------------------------------
 nome          | Introdução a BigData e Analytics                           
 carga_horaria | 36                                                         
-RECORD 1-------------------------------------------------------------------
 nome          | Estatística aplicada                                       
 carga_horaria | 24                                                         
-RECORD 2-------------------------------------------------------------------
 nome          | Visualização de dados e informação                         
 carga_horaria | 24                                                         
-RECORD 3-------------------------------------------------------------------
 nome          | Compartilhamento e segurança de dados                      
 carga_horaria | 24                                                         
-RECORD 4-------------------------------------------------------------------

In [17]:
# Somente 5 registros
df_especializacao.show(n=3, truncate=60)

+----------------------------------+-------------+
|                              nome|carga_horaria|
+----------------------------------+-------------+
|  Introdução a BigData e Analytics|           36|
|              Estatística aplicada|           24|
|Visualização de dados e informação|           24|
+----------------------------------+-------------+
only showing top 3 rows



#### Métodos `describe` e `summary`

O método `describe` computa estatísticas descritivas básicas nas colunas numéricas e textuais. É utilizado em conjunto com o método `show` para exibição do resultado.

**Atenção**: Esta operação pode ser bastante demorada em um DataFrame de maior volume. O motivo ficará claro ao longo da disciplina.

In [18]:
df_especializacao.describe().show(truncate=60)

+-------+----------------------------------+------------------+
|summary|                              nome|     carga_horaria|
+-------+----------------------------------+------------------+
|  count|                                14|                14|
|   mean|                              null|25.714285714285715|
| stddev|                              null|6.4142698058981855|
|    min|           Atividades Integradoras|                12|
|    max|Visualização de dados e informação|                36|
+-------+----------------------------------+------------------+



Já o método `summary` computa algumas estatísticas a mais, os quantis. Sem informar parâmetros, summary irá calcular os quantis 25%, 50% (mediana) e 75%. O parâmetro de `summary` possiblita escolher quais estatísticas serão calculadas.

As estatísticas disponíveis estão descritas na documentação do método ([link](http://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.summary)).

**O mesmo alerta e tempo de processamento segue válido**

In [19]:
df_especializacao.summary().show(truncate=60)

+-------+----------------------------------+------------------+
|summary|                              nome|     carga_horaria|
+-------+----------------------------------+------------------+
|  count|                                14|                14|
|   mean|                              null|25.714285714285715|
| stddev|                              null|6.4142698058981855|
|    min|           Atividades Integradoras|                12|
|    25%|                              null|                24|
|    50%|                              null|                24|
|    75%|                              null|                24|
|    max|Visualização de dados e informação|                36|
+-------+----------------------------------+------------------+



In [20]:
df_especializacao.summary("count", "mean", "10%", "50%", "90%").show()

+-------+----+------------------+
|summary|nome|     carga_horaria|
+-------+----+------------------+
|  count|  14|                14|
|   mean|null|25.714285714285715|
|    10%|null|                24|
|    50%|null|                24|
|    90%|null|                36|
+-------+----+------------------+



#### Método `columns`

Retorna uma lista com os nomes das colunas do DataFrame.

In [21]:
df_especializacao.columns

['nome', 'carga_horaria']

#### Método count

Retorna a quantidade de registros de um DataFrame.

**Atenção**: Por mais que não pareça intuitivo, este operação pode ser bastante demorada em um DataFrame de maior volume, e novamente digo que o motivo ficará claro ao longo da disciplina!

In [22]:
df_especializacao.count()

14

## O caminho contrário

Da mesma forma como conseguimos enviar dados do Python para o Spark (🐍➡️💥) podemos também trazer dados do Spark  para o Python (🐍⬅️💥).

**Mas antes temos que conversar sobre volumes de dados.**

> Neste Objeto de Aprendizagem estamos trabalhando com pequenos volumes de dados em ambiente local, então a transferência de dados não causará dores de cabeça. No entanto, considerem o cenário real de lidar com grandes volumes de dados em um cluster, em ordem de grandeza maior do que sua máquina é capaz de armazenar em memória. Pense em Terabytes (TB) de dados. Tentar transferir este volume de dados do cluster para sua máquina será um desastre.

Na prática, a transferência de DataFrames do Spark para o Python é feita após algum processamento dos dados no Spark. Este processamento pode ser:
- sumarização de dados (estatísticas descritivas, agrupamentos)
- a seleção e filtro de um subconjunto de dados
- amostragem
- etc.

E uma justificativa para transferências deste tipo é a necessidade de uso de recursos que não estão disponíveis no Spark. E mesmo assim temos formas de enviar recursos do Python para uso no Spark (faremos isso em outra oportunidade).

#### Métodos `head`, `first` e `take`

O método `head` retorna o **n** primeiros registros de um DataFrame, retornando somente 1 registro se o parâmetro **n** não for especificado.

Uma pegadinha: Se não especificar o parâmetro, o objeto de retorno é o primeiro registro, de tipo `Row`. No entanto, se especificar **n=1** o retorno será de tipo `list` com o objeto `Row` dentro da lista. `head` sem parâmetros é equivalente ao método `first`.

`take` é bastante similar a `head`, porém com parâmetro **num** obrigatório.

Apesar da aparente confusão, pense que `head` é uma combinação de `first` e `take`:

- `head` sem parâmetro equivale a `first`
- `head` com parâmetro equivale a `take`

In [23]:
um = df_especializacao.head()
lum = df_especializacao.head(n=5)

print((um, type(um)))
print((lum, type(lum)))

(Row(nome='Introdução a BigData e Analytics', carga_horaria=36), <class 'pyspark.sql.types.Row'>)
([Row(nome='Introdução a BigData e Analytics', carga_horaria=36), Row(nome='Estatística aplicada', carga_horaria=24), Row(nome='Visualização de dados e informação', carga_horaria=24), Row(nome='Compartilhamento e segurança de dados', carga_horaria=24), Row(nome='Introdução a Python e linguagem R', carga_horaria=36)], <class 'list'>)


In [24]:
df_especializacao.first()

Row(nome='Introdução a BigData e Analytics', carga_horaria=36)

In [25]:
df_especializacao.take(num=14)

[Row(nome='Introdução a BigData e Analytics', carga_horaria=36),
 Row(nome='Estatística aplicada', carga_horaria=24),
 Row(nome='Visualização de dados e informação', carga_horaria=24),
 Row(nome='Compartilhamento e segurança de dados', carga_horaria=24),
 Row(nome='Introdução a Python e linguagem R', carga_horaria=36),
 Row(nome='Machine Learning', carga_horaria=24),
 Row(nome='Processamento de Alto Desempenho e Aplicações', carga_horaria=24),
 Row(nome='Lidando com BigData: Apache Spark, Hadoop, MapReduce, Hive', carga_horaria=24),
 Row(nome='Gerenciamento e Processamento de grande volume de dados', carga_horaria=24),
 Row(nome='Internet das Coisas e Aplicações Distribuídas', carga_horaria=24),
 Row(nome='Deep Learning', carga_horaria=24),
 Row(nome='Business Intelligence e BigData', carga_horaria=24),
 Row(nome='Atividades Integradoras', carga_horaria=12),
 Row(nome='Preparação para Projeto Aplicado', carga_horaria=36)]

#### Método `collect`

Este método retorna **todos** os registros do DataFrame. 

**Cuidado** ao usar este método com grandes volumes de dados.

In [26]:
df_especializacao.collect()

[Row(nome='Introdução a BigData e Analytics', carga_horaria=36),
 Row(nome='Estatística aplicada', carga_horaria=24),
 Row(nome='Visualização de dados e informação', carga_horaria=24),
 Row(nome='Compartilhamento e segurança de dados', carga_horaria=24),
 Row(nome='Introdução a Python e linguagem R', carga_horaria=36),
 Row(nome='Machine Learning', carga_horaria=24),
 Row(nome='Processamento de Alto Desempenho e Aplicações', carga_horaria=24),
 Row(nome='Lidando com BigData: Apache Spark, Hadoop, MapReduce, Hive', carga_horaria=24),
 Row(nome='Gerenciamento e Processamento de grande volume de dados', carga_horaria=24),
 Row(nome='Internet das Coisas e Aplicações Distribuídas', carga_horaria=24),
 Row(nome='Deep Learning', carga_horaria=24),
 Row(nome='Business Intelligence e BigData', carga_horaria=24),
 Row(nome='Atividades Integradoras', carga_horaria=12),
 Row(nome='Preparação para Projeto Aplicado', carga_horaria=36)]

## Finalizando a sessão

Em muitos casos de uso o Cluster é um ambiente compartilhado e de recursos finitos. Ao concluir o uso de uma sessão do Spark sempre é recomendado finalizá-la para liberar os recursos alocados nesta sessão.

In [28]:
spark.stop()